In [5]:
from imblearn.over_sampling import SMOTE
import pandas as pd
from flaml import AutoML
from sklearn.utils import compute_sample_weight
from tdc import Evaluator
from sklearn.metrics import classification_report

In [ ]:
#Get data
columns_to_drop = ['Drug_ID', 'Drug', 'Y', 'key', 'input']
train_data = pd.read_csv('../../data/DrugTax/train_drugTax_featurized.csv').dropna()
valid_data = pd.read_csv('../../data/DrugTax/valid_drugTax_featurized.csv').dropna()
test_data = pd.read_csv('../../data/DrugTax/test_drugTax_featurized.csv').dropna()

#get splits
X_train, y_train = train_data.drop(columns=columns_to_drop).filter(regex='^(?!char_[.,=#@+\\-\\[\\(\\\\\/])'), train_data['Y']
X_test, y_test = test_data.drop(columns=columns_to_drop).filter(regex='^(?!char_[.,=#@+\\-\\[\\(\\\\\/])'), test_data['Y']
X_valid, y_valid = valid_data.drop(columns=columns_to_drop).filter(regex='^(?!char_[.,=#@+\\-\\[\\(\\\\\/])'), valid_data['Y']

#Use smote to oversample minority class
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

In [ ]:
#Train model using Flaml AutoML

model_config = {
    'task' : 'classification',  # classification 
    'time_budget' : 300,    # time budget in seconds
    'metric' : 'f1', # main metric to be optimized
    'estimator_list' : ['lgbm', 'xgboost', 'rf'] ,
}

model = AutoML()
model.fit(X_res, y_res, **model_config) 

In [6]:
#view the best configuration for each ML model
model.best_config_per_estimator

{'lgbm': {'n_estimators': 86,
  'num_leaves': 66,
  'min_child_samples': 10,
  'learning_rate': 0.4237053714802325,
  'log_max_bin': 10,
  'colsample_bytree': 0.607391031428273,
  'reg_alpha': 0.1352172481077189,
  'reg_lambda': 0.002169425895268579},
 'xgboost': {'n_estimators': 165,
  'max_leaves': 104,
  'min_child_weight': 1.2537688969180918,
  'learning_rate': 0.10531638435811055,
  'subsample': 1.0,
  'colsample_bylevel': 0.6357384957771324,
  'colsample_bytree': 0.47780094418903957,
  'reg_alpha': 0.02922872147319898,
  'reg_lambda': 0.3164947839216637},
 'rf': {'n_estimators': 57,
  'max_features': 0.2240610615176969,
  'max_leaves': 1168,
  'criterion': 'entropy'}}

In [7]:
# Predict test data
y_test_pred = model.predict(X_test)

# Display metrics
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         0.0       0.90      0.94      0.92       981
         1.0       0.55      0.43      0.48       181

    accuracy                           0.86      1162
   macro avg       0.73      0.68      0.70      1162
weighted avg       0.84      0.86      0.85      1162



In [8]:
#Evaluate model performance
from typing import Dict, Any
def evaluate_model(y_true, y_pred_proba, threshold: float = 0.5) -> Dict[str, float]:
    metrics = {
        'ROC-AUC': {'name': 'ROC-AUC', 'kwargs': {}},
        'PR-AUC': {'name': 'PR-AUC', 'kwargs': {}},
        'Accuracy': {'name': 'Accuracy', 'kwargs': {'threshold': threshold}},
        'Precision': {'name': 'Precision', 'kwargs': {'threshold': threshold}},
        'Recall': {'name': 'Recall', 'kwargs': {'threshold': threshold}},
        'F1': {'name': 'F1', 'kwargs': {'threshold': threshold}}
    }
    
    results = {}
    for metric_name, config in metrics.items():
        evaluator = Evaluator(name=config['name'])
        score = evaluator(y_true, y_pred_proba, **config['kwargs'])
        results[metric_name] = score
        print(f"{metric_name}: {score:.4f}")
    
    return results

y_pred_proba = model.predict_proba(X_test)[:, 1]
y_true = y_test

evaluation_results = evaluate_model(y_true, y_pred_proba)

ROC-AUC: 0.8100
PR-AUC: 0.5119
Accuracy: 0.8571
Precision: 0.5540
Recall: 0.4254
F1: 0.4813
